In [23]:
!pip install vaderSentiment

Error processing line 3 of /Users/hau/Anaconda3/anaconda3/lib/python3.7/site-packages/googleapis_common_protos-1.52.0-py3.8-nspkg.pth:

  Traceback (most recent call last):
    File "/Users/hau/Anaconda3/anaconda3/lib/python3.7/site.py", line 168, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 580, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
     |████████████████████████████████| 125 kB 4.3 MB/s eta 0:00:01


In [24]:
import pandas as pd
import re
from nltk.corpus import stopwords
# import en_core_web_sm
from nltk.tokenize import word_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer, SentiText
from selenium import webdriver
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

## Scraper for BeerAdvocate.com

In [4]:
def get_beer_urls():
    driver = webdriver.Chrome('../chromedriver.exe')
    driver.get('https://www.beeradvocate.com/beer/top-rated/')
    table = driver.find_element_by_xpath('//*[@id="ba-content"]/table')
    beers = table.find_elements_by_tag_name('tr')

    text_file = open("beer_urls.txt", "w")


    for beer in beers:
        try:
            url = beer.find_elements_by_tag_name('a')[0].get_attribute('href')
            text_file.write(url + '\n')
        except IndexError:
            print('error')
            continue

    text_file.close()

In [26]:
def get_beer_comments():
    text_file = open('beer_urls.txt', 'r')
    driver = webdriver.Chrome('../chromedriver.exe')
    product_names = []
    product_reviews = []
    user_ratings = []

    for u in text_file:
        url = u.rstrip('\n')
        driver.get(url)
        beer_name = driver.find_element_by_xpath('//*[@id="content"]/div/div/div[3]/div/div/div[1]/h1').text
        review_table = driver.find_element_by_xpath('//*[@id="rating_fullview"]')
        reviews = review_table.find_elements_by_id('rating_fullview_content_2')
        for review in reviews:
            rating = review.find_element_by_class_name('BAscore_norm').text
            review_spans = review.find_elements_by_tag_name('span')
            for span in review_spans:
                driver.execute_script("""
                                    var element = arguments[0];
                                    element.parentNode.removeChild(element);
                                    """, span)
            review_text = review.text

            review_text = review_text.replace('\n', '').strip('  rDev ')
            beer_name = beer_name.split('\n')[0]

            product_reviews.append(review_text)
            product_names.append(beer_name)
            user_ratings.append(rating)

    df = pd.DataFrame({'product_name': product_names, 'product_review': product_reviews, 'user_rating': user_ratings})
    df.to_csv('scraped.csv')

In [27]:
# get_beer_urls()
# get_beer_comments()

## Create word frequency list

In [2]:
def word_frequency(csv_dir, text_column, output_dir):
    df = pd.read_csv(csv_dir, index_col=0)
    frequencies = {}
    for index, row in df.iterrows():
        words = str(row[text_column]).split(' ')
        for word in words:
            if word.lower() not in stopwords.words('english'):
                if word.lower() in frequencies:
                    frequencies[word.lower()] = frequencies[word.lower()] + 1
                else:
                    frequencies[word.lower()] = 1

    freq_df = pd.DataFrame(list(frequencies.items()), columns=['word', 'frequency'])
    freq_df = freq_df.sort_values(by='frequency', ascending=False)

    freq_df.to_csv(output_dir)

In [3]:
# word_frequency('scraped.csv', 'product_review')
freq = pd.read_csv('word_frequency.csv', index_col=0)
freq.sort_values(by='frequency', ascending=False).head(10)

,word,frequency
58,beer,3176
126,-,2641
29,dark,2594
15,like,2277
107,head,2043
394,nice,1886
125,taste,1869
179,one,1791
1008,light,1726
297,sweet,1725


#### Modify raw text to convert similar attributes to one term

In [4]:
words = {
        "crisp": ["carbonation", "effervescent", "crisp"],
        "fruity": ["fruit", "orange", "citrus", "fruity"],
        "robust": ["light", "medium", "strong", "bold", "robust", "rich"],
        "malty": ["malty", "malt", "caramel", "grain"], 
        "texture": ["smooth", "creamy", "texture"],
        "floral": ["floral", "flower", "oak"], 
        "diacetyl": ["butter", "buttery", "butterscotch"],
        "sweet": ["chocolate", "vanilla", "sweet", "sweetness"],
        "bitter": ["bitterness", "bitter"]
    }
    
def replace_synonyms(text, replace_words_dict):
    text = str(text).split(' ')
    for i, word in enumerate(text):
        for k in replace_words_dict:
            if word in replace_words_dict[k]:
                text[i] = k

    return ' '.join(text)


In [5]:
# df = pd.read_csv('scraped.csv', index_col=0)
# df['replaced'] = df['product_review'].apply(lambda x: replace_synonyms(x, words))
# df.to_csv('replaced.csv')

In [6]:
# word_frequency('replaced.csv', 'replaced', 'replaced_frequency.csv')
freq = pd.read_csv('replaced_frequency.csv', index_col=0)
freq.sort_values(by='frequency', ascending=False).head(10)

,word,frequency
50,sweet,4575
42,robust,3609
304,fruity,3477
58,beer,3176
126,-,2641
29,dark,2594
15,like,2277
107,head,2043
390,nice,1886
125,taste,1869


In [7]:
attributes = ['sweet', 'robust', 'fruity']

## Cosine Similarity

In [8]:
def cosine_similarity_df(csv_dir, text_column, attributes):
    df = pd.read_csv(csv_dir, index_col=0)

    def sim(words, attributes):
        documents = [words, ' '.join(attributes)]
        count_vectorizer = CountVectorizer(stop_words='english')
        sparse_matrix = count_vectorizer.fit_transform(documents)
        doc_term_matrix = sparse_matrix.todense()
        df = pd.DataFrame(doc_term_matrix,
                          columns=count_vectorizer.get_feature_names(),
                          index=['x', 'y'])
        return cosine_similarity(df, df)[0, 1]



    df['similarity'] = df[text_column].apply(lambda w: sim(str(w), attributes))
    df.to_csv('similarity.csv')

In [18]:
#cosine_similarity_df('replaced.csv', 'replaced', attributes)
cosine = pd.read_csv('similarity.csv', index_col=0)
cosine[['product_name', 'similarity', 'user_rating']].sort_values(by='similarity', ascending=False).head(5)

,product_name,similarity,user_rating
980,Bourbon County Brand Vanilla Stout,0.476331,4.24
5767,All That Is And All That Ever Will Be,0.474463,4.59
3018,Parabajava,0.468293,4.60
4801,Coconut Vibes,0.450835,4.49
1730,Doppelganger,0.439941,4.55


## User Rating

In [33]:
cosine[['product_name', 'similarity', 'user_rating']].sort_values(by=['user_rating', 'similarity'], ascending=False).head()

,product_name,similarity,user_rating
146,Pliny The Younger,0.408248,5.0
33,Marshmallow Handjee,0.366295,5.0
645,Barrel Aged Imperial German Chocolate Cupcake ...,0.356348,5.0
3858,The Abyss,0.350438,5.0
4454,Double Barrel Jesus,0.284747,5.0


## Vader Sentiment

In [25]:
def sentiment(csv_dir, text_column, attributes):
    analyzer = SentimentIntensityAnalyzer()
    df = pd.read_csv(csv_dir, index_col=0)
    
    def get_sentiment(text, word_padding, word, analyzer):
        text = str(text).split(' ')
        try:
            word_index = text.index(word)
            left_index = max(0, word_index - word_padding)
            right_index = min(len(text), word_index + word_padding)
            string = ' '.join(text[left_index: right_index])
            return analyzer.polarity_scores(string)['compound']
        except ValueError:
            return None
    
    for attr in attributes:
        df[attr] = df[text_column].apply(lambda x: get_sentiment(x, 3, attr, analyzer))

    df.to_csv('sentiment.csv')

In [26]:
sentiment('similarity.csv', 'replaced', attributes)
df = pd.read_csv('sentiment.csv', index_col=0)
df[attributes + ['replaced']].head(10)

,sweet,robust,fruity,replaced
0,NaN,NaN,NaN,Smell: early morning pancakes and coffee befor...
1,0.4588,0.4005,NaN,2019 vintage. Pours a very dark brown color wi...
2,NaN,NaN,NaN,It's hyped... There is a lot of breweries doin...
3,NaN,NaN,NaN,Reviewing 2019 vintage. This pours thick and c...
4,NaN,NaN,NaN,2018 version. Poured dark with a small head. S...
5,0.4588,NaN,NaN,A (very expensive) draft pour at TG on release...
6,NaN,NaN,NaN,"2016 Silver Wax. Aroma has whiskey, maple, tof..."
7,NaN,NaN,NaN,The beer pours Pitch Black with a frothy tan h...
8,0.4588,NaN,NaN,Probably the smoothest beer I have ever had. S...
9,0.7184,NaN,0.4588,"ark black, very thick, a little bit of tan hea..."


In [27]:
df.groupby('product_name').mean().sort_values(by='fruity', ascending=False).head(5)

,user_rating,similarity,sweet,robust,fruity
product_name,,,,,
Coconut B-Bomb,4.4888,0.069990,0.529417,0.344038,0.7184
Genealogy Of Morals - Bourbon Barrel-Aged,4.5548,0.083138,0.542864,0.335909,0.6705
Coffee Cinnamon B-Bomb,4.4632,0.062362,0.525880,0.433550,0.4754
Speedway Stout - Bourbon Barrel-Aged,4.5336,0.084184,0.495592,0.345360,0.4588
Darkstar November,4.5052,0.129275,0.456018,0.437471,0.4588


## Final Recommendations

In [39]:
df = pd.read_csv('sentiment.csv', index_col=0)
df2 = pd.read_csv('similarity.csv', index_col=0)
df['similarity'] = df2['similarity']

average_bow = df.groupby('product_name').mean()
average_bow['avg_feature_sentiment'] = average_bow[attributes].mean(axis=1)

average_bow['evaluation'] = average_bow['avg_feature_sentiment'] + average_bow['similarity']
#average_bow['evaluation'].sort_values(ascending=False).head(5)

In [40]:
average_bow[['user_rating', 'similarity', 'avg_feature_sentiment','evaluation']].\
            sort_values(by = 'evaluation', ascending=False).head(5)

,user_rating,similarity,avg_feature_sentiment,evaluation
product_name,,,,
Mornin' Delight,4.5060,0.100301,0.554860,0.655161
Black Gold,4.4128,0.125335,0.524695,0.650029
Marshmallow Handjee,4.7016,0.140416,0.506178,0.646594
Imperial German Chocolate Cupcake Stout,4.5292,0.192629,0.445565,0.638193
Coconut Vibes,4.5920,0.153014,0.480516,0.633530


We would recommend the Mornin' Delight, Black Gold, or Marshmallow Handjee if the user prefers a sweet, robust, and fruity drink.

## Insights

### Beer Attributes considered - sweet, robust and fruity.

#### Our Recommendation: 
1. Mornin' Delight
2. Black Gold
3. Marshmallow Handjee

By reading through the user reviews for the three products, we found that people love the maple syrup taste and the long-lasting aftertaste in `Mornin' Delight` which backs up the `sweet` preference of our user. The same thing goes to `Black Gold`, which is highly regarded for its chocolate malt, Cake, vanilla and roasty flavor. `Marshmallow Handjee` gets high scores from users who love `fruity` flavor and the hit of citrus and raspberry. All of our three recommendations have a rich mix of fruity and honey flavors which fit perfectly with the user preference of sweet, robust and fruity drinks. 

Other than that, the three recommended beers also have good reputations for the thick and silky monthfeel. `Mornin Delight` has been rated highly for its pillowy carbonation and encroaching sticky texture while `Black Gold` was frequently praised by the lingering and chewy taste. `Marshmallow Handjee` however, was commented by multiple reviewers that it has a sharp carbonation which cuts through its massive body.

#### Top Rates Beers from the dataset:
1. Pliny The Younger
2. Marshmallow Handjee
3. Barrel Aged Imperial German Chocolate Cupcake Stout

We could see there is an overlap between the top rated beers and the recommended beers by our evaluation system: `Marshmallow Handjee`. This demonstrates the massive acception for `Marshmallow Handjee` from sweet and fruity beer lovers. However, for the rest two top rated beers, we didn't find a good match between either of them and the three desired attributes. `Pliny The Younger` is a triple IPA well known by its hoppy aroma and balanced taste with crisp and bitterness. As for `Barrel Aged Imperial German Chocolate Cupcake Stout`, most of its comments regarded its light, spicy, warming barrel appearance in taste as the shining point while barely mentioned `sweet`, `robust` or `fruity`. 

We concluded from there that while looking for beers by user ratings could provide us nice beers with high general quality and unique tastes, this methodology lacks variety and reliable matching ability with features we want.

## Extra
### Comparison to word vectors

In [3]:
def vector_similarity_df(csv_dir, text_column, attributes):
    df = pd.read_csv(csv_dir, index_col=0)
    attributes = ' '.join(attributes)
    nlp = spacy.load('en_core_web_lg')
    def sim(words, attributes):
        doc1 = nlp(words)
        doc2 = nlp(attributes)
        return doc1.similarity(doc2)

    df['similarity'] = df[text_column].apply(lambda w: sim(str(w), attributes))
    df.to_csv('similarity_vector.csv')

In [4]:
def pct_reviews_with_feature(beer, attributes, method):
    df_beer = df[df['product_name'] == beer]
    q = len(df_beer[df_beer['product_review'].str.contains('|'.join(attributes))])
    print(method,' '*(15- len(method)), beer, ' ' * (41 - len(beer)), q, ' ' * (24 - len(str(q))), len(df_beer), ' ' * (13 - len(str(len(df_beer)))), str(round((q / len(df_beer)) * 100, 2)) + '%')

In [7]:
# vector_similarity_df('replaced.csv', 'replaced', attributes)

df = pd.read_csv('sentiment.csv', index_col=0)
df2 = pd.read_csv('similarity_vector.csv', index_col=0)
df['similarity'] = df2['similarity']

average_vector = df.groupby('product_name').mean()
average_vector['avg_feature_sentiment'] = average_vector[attributes].mean(axis=1)

average_vector['evaluation'] = average_vector['avg_feature_sentiment'] + average_vector['similarity']
average_vector['evaluation'].sort_values(ascending=False).head(5)

product_name
Coconut B-Bomb                               1.177848
Mornin' Delight                              1.176185
Genealogy Of Morals - Bourbon Barrel-Aged    1.162127
Black Gold                                   1.157820
Speedway Stout - Vietnamese Coffee           1.140839
Name: evaluation, dtype: float64

Results are just a tad different. Mornin' Delight went up to the top recommendation, Black Gold went down, Coconut B-Bomb emerged as a new primary recommendation.

In [10]:
print('WORD METHOD      BEER NAME                                  # REVIEWS MENTIONING      # TOTAL        PERCENT')
print('                                                            ATTRIBUTE                 REVIEWS        %')

for beer in average_vector['evaluation'].sort_values(ascending=False).head(5).index:
    pct_reviews_with_feature(beer, attributes, 'Word Vector')

for beer in average_bow['evaluation'].sort_values(ascending=False).head(5).index:
    pct_reviews_with_feature(beer, attributes, 'Bag Of Words')

WORD METHOD      BEER NAME                                  # REVIEWS MENTIONING      # TOTAL        PERCENT
                                                            ATTRIBUTE                 REVIEWS        %
Word Vector      Coconut B-Bomb                             12                        25             48.0%
Word Vector      Mornin' Delight                            13                        25             52.0%
Word Vector      Genealogy Of Morals - Bourbon Barrel-Aged  13                        25             52.0%
Word Vector      Black Gold                                 11                        25             44.0%
Word Vector      Speedway Stout - Vietnamese Coffee         10                        25             40.0%
Bag Of Words     Mornin' Delight                            13                        25             52.0%
Bag Of Words     Black Gold                                 11                        25             44.0%
Bag Of Words     Marshmallow Handjee   

### Recommendation system ignoring sentiment and similarity

In [11]:
average_bow.sort_values(by='user_rating', ascending=False).head(5)

,user_rating,similarity,sweet,robust,fruity,avg_feature_sentiment,evaluation
product_name,,,,,,,
Chemtrailmix,4.754286,0.056545,0.489040,0.340000,NaN,0.414520,0.471065
It Was All A Dream,4.747200,0.093836,0.562091,0.427600,0.4588,0.482830,0.576666
SR-71,4.746400,0.071793,0.593760,0.175700,0.2997,0.356387,0.428179
Barrel-Aged Abraxas,4.744400,0.101112,0.489560,0.503554,NaN,0.496557,0.597669
Ann,4.725200,0.029416,0.324600,0.429350,0.1043,0.286083,0.315500


If you ignored the sentiment and similarity scores, the resulting products that have the highest rank would not be sufficient for a recommendation system. A user could have a preference for a unique product with specific attributes that are not represented in the highest rated products. Just because something is popular does not mean it would be a quality recommendation for a user's specified tastes.